In [1]:
import keras
import tensorflow
import matplotlib.pyplot as plt
import numpy as np
from sklearn import preprocessing



Using TensorFlow backend.


# This is level 1 heading

Scenariusz: 
Mając zbiór koszykarzy, z których każego można opisać 5 cechami,
spróbuj ich sklasyfikowac


Cechy:

wzrost     120-240 [centymetrow]
wiek       20-45   [lat]
celnosc    1-100   [jednostek]
siła       1-100   [jednostek]
odległość  100+    [decymetrow]

Celność, siła i wzrost kasują się z odległością i wiekiem, gdyż 
im ktoś jest silniejsz, ma lepsze oko  i jest mlodszy 
tym z większej wysokości pewnie trafi do kosza.
Wzór na szanse trafienie dajacy w miare rzeczywiste rezultaty:

szansa = ((wzrost*sila*celnosc)/(odleglosc^2)-wiek)/10
szansa = (szansa - MIN_SZANSE)/(MAX_SZANSE - MIN_SZANSE) * 1000
     gdzie
MAX_SZANSE = ((MAX_FEATURE^2) *MAX_HEIGHT)/(MIN_RANGE^2) - MIN_AGE
MIN_SZANSE = ((MIN_FEATURE^2) *MIN_HEIGHT)/(MAX_RANGE^2) - MAX_AGE



In [2]:
# Klasa pozwalajaca generowac koszykarzy
import random

MAX_RANGE = 100
MIN_RANGE = 30

MAX_AGE = 45
MIN_AGE = 20

MAX_FEATURE = 100
MIN_FEATURE = 45

MAX_HEIGHT = 240
MIN_HEIGHT = 120

MAX_SZANSE = ((MAX_FEATURE**2) *MAX_HEIGHT)/(MIN_RANGE*MIN_AGE) 
MIN_SZANSE = ((MIN_FEATURE**2) *MIN_HEIGHT)/(MAX_RANGE*MIN_AGE)



class Person:
    def __init__(self):
        self.wiek = random.randint(MIN_AGE,MAX_AGE)
        self.wzrost = random.randint(MIN_HEIGHT,MAX_HEIGHT)
        self.celnosc = random.randint(MIN_FEATURE,MAX_FEATURE)
        self.sila = random.randint(MIN_FEATURE,MAX_FEATURE)
        self.odleglosc = random.randint(MIN_RANGE,MAX_RANGE)
        self.szansa = ((self.wzrost*self.sila*self.celnosc)/(self.odleglosc*self.wiek))
        self.szansa = (self.szansa - MIN_SZANSE)/(MAX_SZANSE - MIN_SZANSE) * 100
        if (random.randint(1,100) <= self.szansa):
            self.trafil = True
        else:
            self.trafil = False
            
    def print_stats(self):
        print("Wzrost:     ", self.wzrost)
        print("Sila:       ", self.sila)
        print("Celnosc:    ", self.celnosc)
        print("Wiek:       ", self.wiek)
        print("Odleglosc:  ", self.odleglosc)
        print("Szansa:     ", self.szansa * 100)
        print("Trafil:     ", self.trafil)
    
        
        
print("Min. szansa:   ", MIN_SZANSE)
print("Max. szansa:   ", MAX_SZANSE)


Min. szansa:    121.5
Max. szansa:    4000.0


In [3]:
# generuj probki do treningu i testow
szanse = []
def generujLudzi(ilosc):
    cechy = []
    czy_trafil = []
    for i in range(1,ilosc):
        osoba = Person()
        cechy.append([osoba.wiek, osoba.odleglosc, osoba.sila, osoba.celnosc,
                     osoba.wzrost])
        szanse.append(osoba.szansa)
        if osoba.trafil == True:
            czy_trafil.append(1)
        else:
            czy_trafil.append(0)
    return (cechy, czy_trafil)

(x_train, y_train) = generujLudzi(100000)
(x_test, y_test) = generujLudzi(10000)

In [4]:
# przekonwertuj na tablica numpy'ja i normalizuj dane
x_train = np.array(x_train)
x_test  = np.array(x_test)
y_train = np.array(y_train)
y_test  = np.array(y_test)
scaler = preprocessing.MinMaxScaler(feature_range=(0,1))
scalled_x_train = scaler.fit_transform(x_train)
scalled_x_test  = scaler.fit_transform(x_test)



C:\Users\DELL\Anaconda3\lib\site-packages\sklearn\utils\validation.py:595: DataConversionWarning: Data with input dtype int32 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)
C:\Users\DELL\Anaconda3\lib\site-packages\sklearn\utils\validation.py:595: DataConversionWarning: Data with input dtype int32 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)


In [5]:
# troche statystyk
print("min wiek:                     ", np.min(x_train[:,0:1]))
print("max wiek:                     ", np.max(x_train[:,0:1]))
print("Sredni wiek:                  ", np.mean(x_train[:,0:1]))
print("1 kwartyl wieku:              ", np.quantile(x_train[:,0:1],0.25))
print("mediana wieku:                ", np.quantile(x_train[:,0:1],0.5))
print("3 kwartyl wieku:              ", np.quantile(x_train[:,0:1],0.75))
print("standardowe odchylenie  wiek: ", np.std(x_train[:,0:1]))
print("-----------------------------------------------------")
print("min  odleglosc:              ", np.min (x_train[:,1:2]))
print("max  odleglosc:              ", np.max (x_train[:,1:2]))
print("Srednia  odleglosc:          ", np.mean (x_train[:,1:2]))
print("1 kwartyl odleglosci:        ", np.quantile(x_train[:,1:2],0.25))
print("mediana odleglosci:          ", np.quantile(x_train[:,1:2],0.5))
print("3 kwartyl odleglosci:        ", np.quantile(x_train[:,1:2],0.75))
print("standardowe odchylenie  odl: ", np.std(x_train[:,1:2]))
print("-----------------------------------------------------")
print("min sila:                     ", np.min (x_train[:,2:3]))
print("max sila:                     ", np.max (x_train[:,2:3]))
print("Srednia sila:                 ", np.mean (x_train[:,2:3]))
print("1 kwartyl sily:               ", np.quantile(x_train[:,2:3],0.25))
print("Mediana sily:                 ", np.quantile(x_train[:,2:3],0.5))
print("3 kwartyl sily:               ", np.quantile(x_train[:,2:3],0.75))
print("standardowe odchylenie  sily: ", np.std(x_train[:,2:3]))
print("-----------------------------------------------------")
print("min celnosc:                     ", np.min (x_train[:,3:4]))
print("max celnosc:                     ", np.max (x_train[:,3:4]))
print("Srednia celnosc:                 ", np.mean (x_train[:,3:4]))
print("1 kwartyl celnosci:              ", np.quantile(x_train[:,3:4],0.25))
print("Mediana celnosci:                ", np.quantile(x_train[:,3:4],0.5))
print("3 kwartyl celnosci:              ", np.quantile(x_train[:,3:4],0.75))
print("standardowe odchylenie  wiek:    ", np.std(x_train[:,3:4]))
print("-----------------------------------------------------")
print("min wzrost:                   ", np.min (x_train[:,4:5]))
print("max wzrost:                   ", np.max (x_train[:,4:5]))
print("Sredni wzrost:                ", np.mean (x_train[:,4:5]))
print("1 kwartyl wieku:              ", np.quantile(x_train[:,4:5],0.25))
print("Mediana wieku:                ", np.quantile(x_train[:,4:5],0.5))
print("3 kwartyl wieku:              ", np.quantile(x_train[:,4:5],0.75))
print("standardowe odchylenie  wiek: ", np.std(x_train[:,4:5]))
print("-----------------------------------------------------")
print("-----------------------------------------------------")
print("min szansa:                    ", np.min (szanse))
print("max szansa:                    ", np.max (szanse))
print("Srednia szansa:                ", np.mean (szanse))
print("1 kwartyl szans:               ", np.quantile(szanse,0.25))
print("Mediana szans:                 ", np.quantile(szanse,0.5))
print("3 kwartyl szans:               ", np.quantile(szanse,0.75))
print("standardowe odchylenie  szans: ", np.std(szanse))

nie = 0
tak = 0

for i in y_train:
    if i == 0:
        nie += 1
    else:
        tak += 1
        
print(tak, nie)

min wiek:                      20
max wiek:                      45
Sredni wiek:                   32.50782507825078
1 kwartyl wieku:               26.0
mediana wieku:                 33.0
3 kwartyl wieku:               39.0
standardowe odchylenie  wiek:  7.4962736216032555
-----------------------------------------------------
min  odleglosc:               30
max  odleglosc:               100
Srednia  odleglosc:           64.96618966189662
1 kwartyl odleglosci:         47.0
mediana odleglosci:           65.0
3 kwartyl odleglosci:         83.0
standardowe odchylenie  odl:  20.513539307623585
-----------------------------------------------------
min sila:                      45
max sila:                      100
Srednia sila:                  72.50991509915099
1 kwartyl sily:                59.0
Mediana sily:                  73.0
3 kwartyl sily:                86.0
standardowe odchylenie  sily:  16.109304587372318
-----------------------------------------------------
min celnosc:      

In [6]:
# stworz model

from keras.layers import Dense

model = keras.Sequential()
model.add(Dense(100, input_shape = (5,), activation='softmax'))
model.add(Dense(100, activation='softmax'))
model.add(Dense(1, activation='sigmoid'))

model.summary()


Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 100)               600       
_________________________________________________________________
dense_2 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 101       
Total params: 10,801
Trainable params: 10,801
Non-trainable params: 0
_________________________________________________________________


In [7]:
model.compile(optimizer="adam", loss="mean_squared_error",
             metrics=['accuracy'])
model.fit(scalled_x_train, y_train, epochs=10, batch_size=1000)
valid_test, valid_acc = model.evaluate(scalled_x_test, y_test)

print("Loss walidacji:        ", valid_test)
print("Poprawnosc Walidacji:  ", valid_acc*100,"%")

Instructions for updating:
Use tf.cast instead.
Epoch 1/10
99999/99999 [==============================] - 1s 11us/step - loss: 0.2323 - acc: 0.8442
Epoch 2/10
99999/99999 [==============================] - 1s 8us/step - loss: 0.1962 - acc: 0.9001
Epoch 3/10
99999/99999 [==============================] - 1s 8us/step - loss: 0.1680 - acc: 0.9001
Epoch 4/10
99999/99999 [==============================] - 1s 8us/step - loss: 0.1464 - acc: 0.9001
Epoch 5/10
99999/99999 [==============================] - 1s 8us/step - loss: 0.1303 - acc: 0.9001
Epoch 6/10
99999/99999 [==============================] - 1s 8us/step - loss: 0.1185 - acc: 0.9001
Epoch 7/10
99999/99999 [==============================] - 1s 9us/step - loss: 0.1098 - acc: 0.9001
Epoch 8/10
99999/99999 [==============================] - 1s 8us/step - loss: 0.1036 - acc: 0.9001
Epoch 9/10
99999/99999 [==============================] - 1s 8us/step - loss: 0.0993 - acc: 0.9001
Epoch 10/10
9999/9999 [==============================] - 0s 